In [2]:
import pandas as pd
import numpy as np
import qcew_2014_2021
import qcew_2010_2013
# import all functions from qcew module
from qcew_2014_2021 import *
from qcew_2010_2013 import *

import msa_codes
from msa_codes import *



In [3]:
FOSSIL_NAICS_CODES = ['2121','211','213','23712','486','4247','22112']
old_years = ['2010','2011','2012','2013']


2014-2021 data (via API)

In [4]:
#FOR FOSSIL

fossil_industry_data = get_2014_industry_data('fossil')

cleaned_fossil_data = get_cleaned_2014_industry_data(fossil_industry_data)

fossil_clean_with_geo = get_2014_industry_with_bls_geographic_tag(cleaned_fossil_data)

aggregated_fossil_data = aggregate_data_by_type(fossil_clean_with_geo,'fossil')



/Users/mariacastillo-coding/Documents/GitHub/rmi-energy-communities/src/energy_comms/extract/qcew_2014_2021.py:94: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['geographic_level'] = np.where(data['area_title'].str.contains('(Combined)'),'aggregated_stat_area',data['geographic_level'])


In [5]:
#FOR TOTALS

total_employment_data = get_2014_industry_data('total')

cleaned_total_data = get_cleaned_2014_industry_data(total_employment_data)

total_clean_with_geo = get_2014_industry_with_bls_geographic_tag(cleaned_total_data)

aggregated_total_data = aggregate_data_by_type(total_clean_with_geo,'total')

# drop ownership and industry column

aggregated_total_data = aggregated_total_data.drop(columns=['own_code','industry_code'])

/Users/mariacastillo-coding/Documents/GitHub/rmi-energy-communities/src/energy_comms/extract/qcew_2014_2021.py:94: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['geographic_level'] = np.where(data['area_title'].str.contains('(Combined)'),'aggregated_stat_area',data['geographic_level'])


In [6]:
# bring together

#fossil

total_fossil_data = aggregated_fossil_data.groupby(['area_fips','area_title','geographic_level','year']).agg({'annual_avg_emplvl':'sum'}).reset_index()

total_fossil_data = total_fossil_data.rename(columns={'annual_avg_emplvl':'fossil_employees'})

#aggregated total

aggregated_total_data = aggregated_total_data.rename(columns={'annual_avg_emplvl':'total_employees'})

In [7]:
# merge on year

combined_attempt = aggregated_total_data.merge(total_fossil_data,on=['area_fips','area_title','geographic_level','year'],how='left')

2010-2013 data (via online downloading)

In [8]:
test = extract_2010()

In [9]:
for yr in old_years:
    download_qcew_zip_files(yr)

In [10]:
data_2010 = make_year_df(2010)
data_2011 = make_year_df(2011)
data_2012 = make_year_df(2012)
data_2013 = make_year_df(2013)

data_dfs = [data_2010,data_2011,data_2012,data_2013]

df = pd.concat(data_dfs,ignore_index=True)


In [11]:
df_geo = get_2014_industry_with_bls_geographic_tag(df)

/Users/mariacastillo-coding/Documents/GitHub/rmi-energy-communities/src/energy_comms/extract/qcew_2014_2021.py:94: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['geographic_level'] = np.where(data['area_title'].str.contains('(Combined)'),'aggregated_stat_area',data['geographic_level'])


In [12]:
#make total df of 2010-2013 data
total_employment = df_geo.query("industry_code == '10' & own_code == 0")
#rename employee column as total for future merge
total_employment = total_employment.rename(columns={'annual_avg_emplvl':'total_employees'})

#keep columns that are in run_qcew.ipynb
total_employment_df = total_employment[['area_fips','area_title','geographic_level','year','total_employees']]


In [13]:
#make fossil employment

FOSSIL_NAICS_CODES = ['2121','211','213','23712','486','4247','22112']


fossil_employment = df_geo.loc[df_geo['industry_code'].isin(FOSSIL_NAICS_CODES)]


# group by all ownership codes

total_fossil_employment = fossil_employment.groupby(['area_fips','area_title','geographic_level','year']).agg({'annual_avg_emplvl':'sum'}).reset_index()

total_fossil_employment = total_fossil_employment.rename(columns={'annual_avg_emplvl':'fossil_employees'})

total_fossil_employment_short = total_fossil_employment[['area_fips','area_title','geographic_level','year','fossil_employees']]


In [14]:
#combine for total and fossil 2010-2013

combined_2010= total_employment_df.merge(total_fossil_employment_short,on=['area_fips','area_title','geographic_level','year'],how='left')
combined_2010['year'] = combined_2010['year'].astype(str)

In [15]:
#group all

combined_dfs = [combined_2010,combined_attempt]

combined = pd.concat(combined_dfs,ignore_index=True)

combined

,area_fips,area_title,geographic_level,year,total_employees,fossil_employees
0,37059,"Davie County, North Carolina",county,2010,9479,0.0
1,49045,"Tooele County, Utah",county,2010,15628,0.0
2,17045,"Edgar County, Illinois",county,2010,6273,0.0
3,45085,"Sumter County, South Carolina",county,2010,34670,8.0
4,55073,"Marathon County, Wisconsin",county,2010,64645,16.0
...,...,...,...,...,...,...
50216,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2017,19449252,NaN
50217,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2018,19662669,NaN
50218,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2019,19771027,NaN
50219,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2020,18904204,NaN


In [16]:
# percent of fossil workers

combined['percent_fossil_employees'] = combined['fossil_employees'] / combined['total_employees'] * 100
combined

,area_fips,area_title,geographic_level,year,total_employees,fossil_employees,percent_fossil_employees
0,37059,"Davie County, North Carolina",county,2010,9479,0.0,0.000000
1,49045,"Tooele County, Utah",county,2010,15628,0.0,0.000000
2,17045,"Edgar County, Illinois",county,2010,6273,0.0,0.000000
3,45085,"Sumter County, South Carolina",county,2010,34670,8.0,0.023075
4,55073,"Marathon County, Wisconsin",county,2010,64645,16.0,0.024751
...,...,...,...,...,...,...,...
50216,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2017,19449252,NaN,NaN
50217,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2018,19662669,NaN,NaN
50218,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2019,19771027,NaN,NaN
50219,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2020,18904204,NaN,NaN


In [17]:
#geoid processing

# clean area fips column first
combined['area_fips'] = combined['area_fips'].astype(str)

combined['area_fips'] = combined['area_fips'].apply(lambda x:str(x).zfill(5))

combined['geoid'] = combined['area_fips']


combined

,area_fips,area_title,geographic_level,year,total_employees,fossil_employees,percent_fossil_employees,geoid
0,37059,"Davie County, North Carolina",county,2010,9479,0.0,0.000000,37059
1,49045,"Tooele County, Utah",county,2010,15628,0.0,0.000000,49045
2,17045,"Edgar County, Illinois",county,2010,6273,0.0,0.000000,17045
3,45085,"Sumter County, South Carolina",county,2010,34670,8.0,0.023075,45085
4,55073,"Marathon County, Wisconsin",county,2010,64645,16.0,0.024751,55073
...,...,...,...,...,...,...,...,...
50216,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2017,19449252,NaN,NaN,USNMS
50217,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2018,19662669,NaN,NaN,USNMS
50218,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2019,19771027,NaN,NaN,USNMS
50219,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2020,18904204,NaN,NaN,USNMS


In [18]:
# take out C in MSA to add extra 0 

combined['geoid'] = combined['geoid'].str.replace('C','')
combined['geoid'] = np.where(combined['geographic_level']=='metropolitan_stat_area',combined['geoid'] + '0',combined['geoid'])

# only keep geo level we need

final_employment = combined.query("geographic_level=='county' or geographic_level=='metropolitan_stat_area'")


In [39]:
# bring in BLS MSA crosswalk


msa = msa_codes.extract()

msa['geoid'] = msa['geoid'].astype(str)
employment = final_employment.merge(msa,on='geoid',how='inner')

make threshold criteria

In [76]:
employment

,area_fips,area_title,geographic_level,year,total_employees,fossil_employees,percent_fossil_employees,geoid,FIPS code,State,State abbreviation,May 2021 MSA code,May 2021 MSA name,County code,Township code,County name (or Township name for the New England states)
0,17045,"Edgar County, Illinois",county,2010,6273,0.0,0.0,17045,17,Illinois,IL,1700003,East Central Illinois nonmetropolitan area,045,000,Edgar County
1,17045,"Edgar County, Illinois",county,2011,6269,0.0,0.0,17045,17,Illinois,IL,1700003,East Central Illinois nonmetropolitan area,045,000,Edgar County
2,17045,"Edgar County, Illinois",county,2012,6324,0.0,0.0,17045,17,Illinois,IL,1700003,East Central Illinois nonmetropolitan area,045,000,Edgar County
3,17045,"Edgar County, Illinois",county,2013,6450,0.0,0.0,17045,17,Illinois,IL,1700003,East Central Illinois nonmetropolitan area,045,000,Edgar County
4,17045,"Edgar County, Illinois",county,2014,6717,0.0,0.0,17045,17,Illinois,IL,1700003,East Central Illinois nonmetropolitan area,045,000,Edgar County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50188,46102,"Oglala Lakota County, South Dakota",county,2017,3703,NaN,NaN,46102,46,South Dakota,SD,4600003,West South Dakota nonmetropolitan area,102,000,Oglala Lakota County
50189,46102,"Oglala Lakota County, South Dakota",county,2018,3688,NaN,NaN,46102,46,South Dakota,SD,4600003,West South Dakota nonmetropolitan area,102,000,Oglala Lakota County
50190,46102,"Oglala Lakota County, South Dakota",county,2019,3688,NaN,NaN,46102,46,South Dakota,SD,4600003,West South Dakota nonmetropolitan area,102,000,Oglala Lakota County
50191,46102,"Oglala Lakota County, South Dakota",county,2020,3898,NaN,NaN,46102,46,South Dakota,SD,4600003,West South Dakota nonmetropolitan area,102,000,Oglala Lakota County


In [77]:
employment['percent_fossil_employment'] = employment['fossil_employees'] / employment['total_employees'] * 100

In [78]:
employment['meets_fossil_threshold'] = np.where(employment['percent_fossil_employment'] > .17,1,0)
employment

,area_fips,area_title,geographic_level,year,total_employees,fossil_employees,percent_fossil_employees,geoid,FIPS code,State,State abbreviation,May 2021 MSA code,May 2021 MSA name,County code,Township code,County name (or Township name for the New England states),percent_fossil_employment,meets_fossil_threshold
0,17045,"Edgar County, Illinois",county,2010,6273,0.0,0.0,17045,17,Illinois,IL,1700003,East Central Illinois nonmetropolitan area,045,000,Edgar County,0.0,0
1,17045,"Edgar County, Illinois",county,2011,6269,0.0,0.0,17045,17,Illinois,IL,1700003,East Central Illinois nonmetropolitan area,045,000,Edgar County,0.0,0
2,17045,"Edgar County, Illinois",county,2012,6324,0.0,0.0,17045,17,Illinois,IL,1700003,East Central Illinois nonmetropolitan area,045,000,Edgar County,0.0,0
3,17045,"Edgar County, Illinois",county,2013,6450,0.0,0.0,17045,17,Illinois,IL,1700003,East Central Illinois nonmetropolitan area,045,000,Edgar County,0.0,0
4,17045,"Edgar County, Illinois",county,2014,6717,0.0,0.0,17045,17,Illinois,IL,1700003,East Central Illinois nonmetropolitan area,045,000,Edgar County,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50188,46102,"Oglala Lakota County, South Dakota",county,2017,3703,NaN,NaN,46102,46,South Dakota,SD,4600003,West South Dakota nonmetropolitan area,102,000,Oglala Lakota County,NaN,0
50189,46102,"Oglala Lakota County, South Dakota",county,2018,3688,NaN,NaN,46102,46,South Dakota,SD,4600003,West South Dakota nonmetropolitan area,102,000,Oglala Lakota County,NaN,0
50190,46102,"Oglala Lakota County, South Dakota",county,2019,3688,NaN,NaN,46102,46,South Dakota,SD,4600003,West South Dakota nonmetropolitan area,102,000,Oglala Lakota County,NaN,0
50191,46102,"Oglala Lakota County, South Dakota",county,2020,3898,NaN,NaN,46102,46,South Dakota,SD,4600003,West South Dakota nonmetropolitan area,102,000,Oglala Lakota County,NaN,0


calculating who meets employment threshold 

bring in unemployment data to see who meets unemployment threshold

In [80]:
unemployment = pd.read_csv('/Users/mariacastillo-coding/Documents/GitHub/rmi-energy-communities/notebooks/files/unemployment.csv',dtype={'geoid':'str','year':'str','state_code':'str'})

# make it a float for comparison
unemployment['local_area_unemployment'] = pd.to_numeric(unemployment['local_area_unemployment'],errors='coerce')

#fix geoid for merge with employment

unemployment['geoid'] = np.where(unemployment['geographic_level']=='county',unemployment['state_code'] + unemployment['geoid'],unemployment['geoid'].str[0:5])

In [81]:
employment_both = employment.merge(unemployment,on=['geoid','geographic_level','year'],how='inner')

In [82]:
employment_both['meets_unemployment_criteria'] = np.where(employment_both['local_area_unemployment'] > employment_both['national_unemployment_rate_prev_year'],1,0)
employment_both

,area_fips,area_title,geographic_level,year,total_employees,fossil_employees,percent_fossil_employees,geoid,FIPS code,State,...,Township code,County name (or Township name for the New England states),percent_fossil_employment,meets_fossil_threshold,Unnamed: 0,area_text,state_code,local_area_unemployment,national_unemployment_rate_prev_year,meets_unemployment_criteria
0,17045,"Edgar County, Illinois",county,2010,6273,0.0,0.0,17045,17,Illinois,...,000,Edgar County,0.0,0,3080,"Edgar County, IL",17,11.3,9.3,1
1,17045,"Edgar County, Illinois",county,2011,6269,0.0,0.0,17045,17,Illinois,...,000,Edgar County,0.0,0,3081,"Edgar County, IL",17,10.5,9.6,1
2,17045,"Edgar County, Illinois",county,2012,6324,0.0,0.0,17045,17,Illinois,...,000,Edgar County,0.0,0,3082,"Edgar County, IL",17,9.7,8.9,1
3,17045,"Edgar County, Illinois",county,2013,6450,0.0,0.0,17045,17,Illinois,...,000,Edgar County,0.0,0,3083,"Edgar County, IL",17,9.4,8.1,1
4,17045,"Edgar County, Illinois",county,2014,6717,0.0,0.0,17045,17,Illinois,...,000,Edgar County,0.0,0,3084,"Edgar County, IL",17,7.2,7.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50188,46102,"Oglala Lakota County, South Dakota",county,2017,3703,NaN,NaN,46102,46,South Dakota,...,000,Oglala Lakota County,NaN,0,30132,"Oglala Lakota County, SD",46,10.7,4.9,1
50189,46102,"Oglala Lakota County, South Dakota",county,2018,3688,NaN,NaN,46102,46,South Dakota,...,000,Oglala Lakota County,NaN,0,30133,"Oglala Lakota County, SD",46,9.6,4.4,1
50190,46102,"Oglala Lakota County, South Dakota",county,2019,3688,NaN,NaN,46102,46,South Dakota,...,000,Oglala Lakota County,NaN,0,30134,"Oglala Lakota County, SD",46,8.4,3.9,1
50191,46102,"Oglala Lakota County, South Dakota",county,2020,3898,NaN,NaN,46102,46,South Dakota,...,000,Oglala Lakota County,NaN,0,40974,"Oglala Lakota County, SD",46,9.2,3.7,1


In [85]:
employment_both.query("meets_fossil_threshold==1 & meets_unemployment_criteria==1")

,area_fips,area_title,geographic_level,year,total_employees,fossil_employees,percent_fossil_employees,geoid,FIPS code,State,...,Township code,County name (or Township name for the New England states),percent_fossil_employment,meets_fossil_threshold,Unnamed: 0,area_text,state_code,local_area_unemployment,national_unemployment_rate_prev_year,meets_unemployment_criteria
15,21085,"Grayson County, Kentucky",county,2013,7525,22.0,0.292359,21085,21,Kentucky,...,000,Grayson County,0.292359,1,5173,"Grayson County, KY",21,10.2,8.1,1
16,21085,"Grayson County, Kentucky",county,2014,7506,29.0,0.386358,21085,21,Kentucky,...,000,Grayson County,0.386358,1,5174,"Grayson County, KY",21,8.4,7.4,1
17,21085,"Grayson County, Kentucky",county,2015,7613,33.0,0.433469,21085,21,Kentucky,...,000,Grayson County,0.433469,1,23245,"Grayson County, KY",21,6.6,6.2,1
18,21085,"Grayson County, Kentucky",county,2016,7736,32.0,0.413650,21085,21,Kentucky,...,000,Grayson County,0.413650,1,23246,"Grayson County, KY",21,6.6,5.3,1
20,21085,"Grayson County, Kentucky",county,2018,7929,18.0,0.227015,21085,21,Kentucky,...,000,Grayson County,0.227015,1,23248,"Grayson County, KY",21,4.8,4.4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50145,C2942,"Lake Havasu City-Kingman, AZ MSA",metropolitan_stat_area,2016,47595,97.0,0.203803,29420,04,Arizona,...,000,Mohave County,0.203803,1,34246,"Lake Havasu City-Kingman, AZ Metropolitan Stat...",04,6.7,5.3,1
50146,C2942,"Lake Havasu City-Kingman, AZ MSA",metropolitan_stat_area,2017,48983,96.0,0.195986,29420,04,Arizona,...,000,Mohave County,0.195986,1,34247,"Lake Havasu City-Kingman, AZ Metropolitan Stat...",04,6.0,4.9,1
50147,C2942,"Lake Havasu City-Kingman, AZ MSA",metropolitan_stat_area,2018,50290,92.0,0.182939,29420,04,Arizona,...,000,Mohave County,0.182939,1,34248,"Lake Havasu City-Kingman, AZ Metropolitan Stat...",04,5.7,4.4,1
50148,C2942,"Lake Havasu City-Kingman, AZ MSA",metropolitan_stat_area,2019,51173,99.0,0.193461,29420,04,Arizona,...,000,Mohave County,0.193461,1,34249,"Lake Havasu City-Kingman, AZ Metropolitan Stat...",04,5.8,3.9,1


In [86]:
employment_both.columns

Index(['area_fips', 'area_title', 'geographic_level', 'year',
       'total_employees', 'fossil_employees', 'percent_fossil_employees',
       'geoid', 'FIPS code', 'State', 'State abbreviation',
       'May 2021 MSA code ', 'May 2021 MSA name', 'County code',
       'Township code',
       'County name (or Township name for the New England states)',
       'percent_fossil_employment', 'meets_fossil_threshold', 'Unnamed: 0',
       'area_text', 'state_code', 'local_area_unemployment',
       'national_unemployment_rate_prev_year', 'meets_unemployment_criteria'],
      dtype='object')

In [94]:
employment_both = employment_both.rename(columns={'May 2021 MSA name':'msa_name'})

In [98]:
underlying_employment_data = employment_both[['area_title','geographic_level','State','msa_name','geoid','percent_fossil_employees','meets_fossil_threshold','local_area_unemployment','national_unemployment_rate_prev_year','meets_unemployment_criteria']]

underlying_employment_data.to_csv('/Users/mariacastillo-coding/Documents/Github/rmi-energy-communities/notebooks/files/underlying_employment_data.csv')

list of qualified employment areas

In [103]:
qualifying_employment_areas = underlying_employment_data.query("meets_fossil_threshold==1 & meets_unemployment_criteria==1")

qualifying_employment_areas.to_csv('/Users/mariacastillo-coding/Documents/Github/rmi-energy-communities/notebooks/files/qualifying_employment_areas.csv')

In [101]:
qualifying_employment_areas['State'].unique()

array(['Kentucky', 'Michigan', 'Georgia', 'Alaska', 'Montana',
       'Pennsylvania', 'Texas', 'New York', 'West Virginia', 'Louisiana',
       'Alabama', 'Maine', 'Mississippi', 'Utah', 'Illinois', 'Missouri',
       'Ohio', 'Indiana', 'North Carolina', 'South Carolina', 'Oklahoma',
       'Nevada', 'Colorado', 'Washington', 'Florida', 'Wyoming',
       'North Dakota', 'Hawaii', 'California', 'Iowa', 'Kansas', 'Oregon',
       'Virginia', 'Tennessee', 'Arizona', 'Wisconsin', 'Maryland',
       'Arkansas', 'Idaho', 'New Mexico', 'Nebraska', 'Minnesota',
       'Vermont', 'Puerto Rico', 'New Jersey', 'South Dakota',
       'New Hampshire'], dtype=object)